<a href="https://colab.research.google.com/github/cmzwolf/machineLearning/blob/main/ChemTrans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.api._v2.keras import activations

import numpy as np
import matplotlib.pyplot as plt
from importlib import reload

In [ ]:
nbPhisParam = 10
seqMaxLenght = 3
latentDim = 4

NbOfTransBlocks = 6

In [ ]:
a = np.random.rand(seqMaxLenght , nbPhisParam)
a

array([[0.749231  , 0.33566701, 0.88163603, 0.87155149, 0.39412449,
        0.86229768, 0.5528675 , 0.5619276 , 0.28560993, 0.54102378],
       [0.29122267, 0.28675602, 0.23609739, 0.93355352, 0.39487143,
        0.44475151, 0.74033767, 0.30448359, 0.48433186, 0.2384493 ],
       [0.85183999, 0.51789701, 0.55654204, 0.22007827, 0.16092495,
        0.67228727, 0.13069588, 0.52211487, 0.9986423 , 0.3305903 ]])

In [ ]:
# starting the definition of the Transormers model
input = keras.layers.Input(shape = (seqMaxLenght, nbPhisParam))

masked = keras.layers.Masking(mask_value=-1.0, input_shape=(seqMaxLenght, nbPhisParam))(input)
#masked = keras.layers.Masking(mask_value=-1.0)(input)


# embed1, 2 and 3 are embedding layers to convert inputs into a latent space of 
# dimension latentDim

embed1 = keras.layers.Dense(nbPhisParam, activation="relu")(masked)
embed2 = keras.layers.Dense(nbPhisParam, activation="relu")(embed1)
embed3 = keras.layers.Dense(latentDim, activation="relu")(embed2)

transformerInput = embed3

# begin of the loop to define the set of transformer-blocks
for i in range(NbOfTransBlocks):
  # Starting the i-th transformer, from the transformersInput layer
  K = keras.layers.Dense(latentDim)(transformerInput)
  Q = keras.layers.Dense(latentDim)(transformerInput)
  V = keras.layers.Dense(latentDim)(transformerInput)

  # defining the multihead attention mechanisms
  mha = keras.layers.MultiHeadAttention(num_heads=2, key_dim=2)
  attention_output = mha(query = Q, value = V, key = K, use_causal_mask = True)
  
  residual_connection = keras.layers.Add()([transformerInput, attention_output])
  normalized_rc = keras.layers.LayerNormalization(epsilon = 1e-6)(residual_connection)

  #defining the feedForward layer
  fw1 = keras.layers.Dense(latentDim, activation="relu")(normalized_rc)
  fw2 = keras.layers.Dense(latentDim, activation= "relu")(fw1)
  fw3 = keras.layers.Dense(latentDim)(fw2)
  
  residual_connection2 = keras.layers.Add()([normalized_rc, fw3])
  normalized_rc2 = keras.layers.LayerNormalization(epsilon = 1e-6)(residual_connection2)
  # end of the transformers block

  #the output of the (N-1) transformers block will be the input for the N block
  transformerInput = normalized_rc2

# building the head of the transformers with some dense layers
head1 = keras.layers.Dense(latentDim, activation="relu")(transformerInput)
head2 = keras.layers.Dense(nbPhisParam, activation="relu")(head1)
head3 = keras.layers.Dense(nbPhisParam, activation=None)(head2)


model = keras.Model(input, head3)

 


In [ ]:

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 3, 10)]      0           []                               
                                                                                                  
 masking (Masking)              (None, 3, 10)        0           ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 3, 10)        110         ['masking[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 3, 10)        110         ['dense[0][0]']                  
                                                                                              

In [ ]:
a1 = np.random.rand(1,seqMaxLenght , nbPhisParam)


In [ ]:
a1[:, 1:, :]=-1.0
a1

array([[[ 0.50070636,  0.05398985,  0.7107876 ,  0.67811944,
          0.83659072,  0.74536643,  0.32726702,  0.16959711,
          0.55207458,  0.96348064],
        [-1.        , -1.        , -1.        , -1.        ,
         -1.        , -1.        , -1.        , -1.        ,
         -1.        , -1.        ],
        [-1.        , -1.        , -1.        , -1.        ,
         -1.        , -1.        , -1.        , -1.        ,
         -1.        , -1.        ]]])

In [ ]:

b = model(a1)
b

<tf.Tensor: shape=(1, 3, 10), dtype=float32, numpy=
array([[[ 0.6031983 , -0.25286838, -0.07161084,  0.5325496 ,
          0.64164317,  0.35254112, -0.01343392,  0.09527819,
         -0.6869054 ,  0.3321216 ],
        [ 0.38353884,  0.13234004, -0.65850896,  0.25031245,
          0.26480168,  0.07271829, -0.1877878 ,  0.13173133,
         -1.0640119 , -0.41256213],
        [ 0.38353884,  0.13234004, -0.65850896,  0.25031245,
          0.26480168,  0.07271829, -0.1877878 ,  0.13173133,
         -1.0640119 , -0.41256213]]], dtype=float32)>